In [6]:
from csv import reader
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from collections import OrderedDict
from nltk.stem import PorterStemmer
import math
import numpy as np
from numpy.linalg import norm
import csv

In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/mohit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mohit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mohit/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mohit/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
articles, links, titles = [], [], []

with open('articles.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        articles.append(row[5])
        links.append(row[3])
        titles.append(row[4])

# import csv

# with open('titles.csv','w') as result_file:
#     wr = csv.writer(result_file, dialect='excel')
#     wr.writerow(titles)
    
    
# with open('links.csv','w') as result_file:
#     wr = csv.writer(result_file, dialect='excel')
#     wr.writerow(links)

In [12]:
t, av = [], 0
with open('processed_articles.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    for row in csv_reader:
        t.append(row)
        av+= len(row)
        
av /= len(t)
av

1046.246290801187

In [16]:
tokens = [word_tokenize(article) for article in articles]
tokens = [[word for word in article if word.isalpha()] for article in tokens] 
ps = PorterStemmer()

stem_tokens = [[ps.stem(word) for word in article] for article in tokens]
tokens = [[word for word in article if not word in stopwords.words("english")] for article in stem_tokens]


# with open('processed_articles.csv','w') as result_file:
#     wr = csv.writer(result_file, dialect='excel')
#     wr.writerow(tokens)
    
with open("processed_articles.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(tokens)

## Preprocessing

In [ ]:
tokens = [word_tokenize(article) for article in articles]
tokens = [[word for word in article if word.isalpha()] for article in tokens] 
ps = PorterStemmer()

stem_tokens = [[ps.stem(word) for word in article] for article in tokens]
tokens = [[word for word in article if not word in stopwords.words("english")] for article in stem_tokens]

print(f'Size of the index terms - {len(set([token for article in tokens for token in article]))}')


avg_doc_len = 0
for doc in tokens:
    avg_doc_len += len(doc)

avg_doc_len /= 100

sample_query = 'Machine Learning can involve neural networks'

def preprocess_query(query):
    query_terms = query.split(" ")  #Split the query -> tokenisation

    #Query normalisation
    query_terms = [term for term in query_terms if term.isalpha()]
    query_terms = [ps.stem(term) for term in query_terms]
    query_terms = [word for word in query_terms if not word in stopwords.words("english")]

    return query_terms

preprocess_query(sample_query)

Size of the index terms - 6089


['machin', 'learn', 'involv', 'neural', 'network']

In [ ]:
def calculate_combination_factor(K1=1, b=0.3):
    combination_factor = {}
    for word in index_terms:
        combination_factor[word] = [0] * N
        for id, article in enumerate(tokens):
            freq = article.count(word)
            num = (K1 + 1) * freq
            den = (K1 * ((1-b) + (b * (len(article) / avg_doc_len)))) + freq
            combination_factor[word][id] = num / den

    return combination_factor

B_i_j = calculate_combination_factor(K1=1, b=0.3)

In [ ]:
def calculate_BM25(query):
    query_terms = preprocess_query(query)

    scores = []
    for doc_id, document in enumerate(tokens):
        k = set(query_terms) & set(document)

        this_doc__score = 0
        for k_i in k:
            n_i = document_frequency[k_i]
            log_value = math.log(((N - n_i + 0.5) / (n_i + 0.5)), 2)
            this_doc__score += B_i_j[k_i][doc_id] * log_value
        scores.append(this_doc__score)

    ranked_list = np.argsort(scores).tolist()
    # ranked_list.reverse()
    return ranked_list


ranked_documents_BM25 = calculate_BM25(sample_query)
print(ranked_documents_BM25)

[61, 46, 68, 57, 15, 79, 3, 77, 60, 18, 98, 58, 37, 13, 36, 14, 56, 97, 17, 66, 12, 30, 99, 67, 19, 83, 53, 64, 6, 90, 16, 65, 50, 93, 9, 51, 25, 2, 49, 38, 88, 4, 96, 84, 28, 48, 62, 74, 85, 78, 47, 73, 42, 80, 43, 11, 92, 8, 72, 22, 29, 44, 24, 82, 41, 91, 7, 55, 1, 70, 81, 94, 59, 95, 10, 5, 89, 20, 75, 54, 87, 45, 27, 71, 86, 0, 69, 32, 76, 52, 63, 23, 39, 31, 35, 21, 26, 33, 34, 40]


In [10]:
summarised_articles = []
with open('summarised_articles.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    for row in csv_reader:
        summarised_articles.append(row)

len(summarised_articles)

337

In [14]:
unique_docs = set()
u_articles, u_titles, u_links, u_tokens, u_summarised = [], [], [], [], []
for idx, title in enumerate(titles):
    if title not in unique_docs:
        unique_docs.add(title)
        u_articles.append(articles[idx])
        u_titles.append(titles[idx])
        u_links.append(links[idx])
        u_tokens.append(t[idx])
        u_summarised.append(summarised_articles[idx])
        
len(unique_docs), len(u_articles), len(u_titles), len(u_tokens)

(230, 230, 230, 230)

In [16]:
import rank_bm25 

bm25 = rank_bm25.BM25Okapi(u_tokens)


In [44]:
sample_query = 'chatbots are taking over'
ps = PorterStemmer()

def preprocess_query(query):
    query_terms = query.split(" ")  #Split the query -> tokenisation

    #Query normalisation
    query_terms = [term for term in query_terms if term.isalpha()]
    query_terms = [ps.stem(term) for term in query_terms]
    query_terms = [word for word in query_terms if not word in stopwords.words("english")]

    return query_terms

q = preprocess_query(sample_query)

In [45]:
docs = bm25.get_top_n(q, u_articles, n=15)

In [46]:
print(docs[0])

Oh, how the headlines blared:
Chatbots were The Next Big Thing.
Our hopes were sky high. Bright-eyed and bushy-tailed, the industry was ripe for a new era of innovation: it was time to start socializing with machines.
And why wouldn’t they be? All the road signs pointed towards insane success.
At the Mobile World Congress 2017, chatbots were the main headliners. The conference organizers cited an ‘overwhelming acceptance at the event of the inevitable shift of focus for brands and corporates to chatbots’.
In fact, the only significant question around chatbots was who would monopolize the field, not whether chatbots would take off in the first place:
One year on, we have an answer to that question.
No.
Because there isn’t even an ecosystem for a platform to dominate.
Chatbots weren’t the first technological development to be talked up in grandiose terms and then slump spectacularly.
The age-old hype cycle unfolded in familiar fashion...
Expectations built, built, and then..... It all ki

In [50]:
a = np.array(bm25.get_scores(q))
a = a.argsort().tolist()
a.reverse()
u_articles[a[0]]

"Oh, how the headlines blared:\nChatbots were The Next Big Thing.\nOur hopes were sky high. Bright-eyed and bushy-tailed, the industry was ripe for a new era of innovation: it was time to start socializing with machines.\nAnd why wouldn’t they be? All the road signs pointed towards insane success.\nAt the Mobile World Congress 2017, chatbots were the main headliners. The conference organizers cited an ‘overwhelming acceptance at the event of the inevitable shift of focus for brands and corporates to chatbots’.\nIn fact, the only significant question around chatbots was who would monopolize the field, not whether chatbots would take off in the first place:\nOne year on, we have an answer to that question.\nNo.\nBecause there isn’t even an ecosystem for a platform to dominate.\nChatbots weren’t the first technological development to be talked up in grandiose terms and then slump spectacularly.\nThe age-old hype cycle unfolded in familiar fashion...\nExpectations built, built, and then...